In [1]:
import math
import numpy as np
import pandas as pd

from collections import OrderedDict
from ipynb.fs.full.config import Config
from ipynb.fs.full.genco import CPGenco
from ipynb.fs.full.auctioneer import clearing_mechanism, plot

import sys  
sys.path.insert(1, '/home/sanjay/Research/MCTS/Codes/pda_simulator/notebooks/brokers')
from ipynb.fs.full.zi import ZI

import warnings
warnings.filterwarnings('ignore')

In [2]:
name_of_sellers = ['cp_genco']
name_of_buyers = ['buyer1', 'buyer2', 'buyer3']

list_of_sellers = dict()
list_of_buyers = dict()

config = Config()

for seller in name_of_sellers:
    seller_obj = CPGenco("cp_genco")
    list_of_sellers.update({seller: seller_obj})
    
buyer1 = ZI(0, 100, config.market_demand*0.3, 5, name_of_buyers[0])
buyer2 = ZI(0, 100, config.market_demand*0.3, 5, name_of_buyers[1])
buyer3 = ZI(0, 100, config.market_demand*0.4, 5, name_of_buyers[2])

list_of_buyers.update({name_of_buyers[0]: buyer1})
list_of_buyers.update({name_of_buyers[1]: buyer2})
list_of_buyers.update({name_of_buyers[2]: buyer3})

In [3]:
# PDA simulator

rounds = 24
cur_round = 0

while(cur_round < rounds):
    
    proximity = rounds - cur_round    
    
    # asks dataframe
    asks_df = pd.DataFrame(list_of_sellers['cp_genco'].asks(), columns=['ID', 'Price', 'Quantity'])

    # bids dataframe
    if proximity == 24:
        bids_df = pd.DataFrame([["miso", -1e9, np.random.normal(800, 100)]], columns=['ID', 'Price', 'Quantity'])
    else:
        bids_df = pd.DataFrame(columns=['ID', 'Price', 'Quantity'])
        
    for buyer in list_of_buyers.keys():
        buyer_df = pd.DataFrame(list_of_buyers[buyer].bids(), columns=['ID', 'Price', 'Quantity'])
        bids_df = pd.concat([bids_df,buyer_df], ignore_index=True)

    bids_df = bids_df.sort_values(by=['Price'])
            
    # market clearing
    mcp, mcq, cleared_asks_df, cleared_bids_df = clearing_mechanism(asks_df, bids_df)
    
    # update the cleared quantity of sellers
    for seller in list_of_sellers.keys():
        temp = cleared_asks_df.groupby('ID')
        if seller in temp.groups.keys():
            seller_cq = temp.sum()['Quantity'][seller]
            list_of_sellers[seller].set_cleared_quantity(-seller_cq)
        
    # update the cleared quantity of buyers
    for buyer in list_of_buyers.keys():
        temp = cleared_bids_df.groupby('ID')
        if buyer in temp.groups.keys():
            buyer_cq = temp.sum()['Quantity'][buyer]
            list_of_buyers[buyer].set_cleared_demand(buyer_cq)
    
    print('\n----------At Proxomity ', proximity, '------\n')
    print('MCP', mcp)
    print('MCQ', mcq)
    
    cur_round += 1


----------At Proxomity  24 ------

MCP 44.987935707051065
MCQ 880.6075044918022

----------At Proxomity  23 ------

MCP 50.51129426678047
MCQ 15.95107132146073

----------At Proxomity  22 ------

MCP 51.67899912843632
MCQ 14.056881602037269

----------At Proxomity  21 ------

MCP 59.39830248963164
MCQ 2.8711928378629326

----------At Proxomity  20 ------

MCP 61.39972387882033
MCQ 3.3377616740156597

----------At Proxomity  19 ------

MCP 56.74904892911397
MCQ 1.0766973141986016

----------At Proxomity  18 ------

MCP 56.05322062904001
MCQ 1.4815355043372733

----------At Proxomity  17 ------

MCP 57.546288124930086
MCQ 0.7304314579523307

----------At Proxomity  16 ------

MCP 58.24047741323575
MCQ 0.22845684895308582

----------At Proxomity  15 ------

MCP 61.645914290305235
MCQ 0.09938273958123432

----------At Proxomity  14 ------

MCP 40.0
MCQ 0.0

----------At Proxomity  13 ------

MCP 73.72617572126794
MCQ 0.010584365277497908

----------At Proxomity  12 ------

MCP 75.00122232